In [30]:
from azure.identity import DefaultAzureCredential, InteractiveBrowserCredential
from azure.ai.ml import MLClient
from azure.ai.ml.entities import AmlCompute

In [35]:
try:
    credential=DefaultAzureCredential()
    credential.get_token('https://management.azure.com/.default')
except Exception as ex : 
    credential=InteractiveBrowserCredential()
ml_client=MLClient.from_config(credential=credential)



Found the config file in: /config.json


In [36]:
cluster = AmlCompute(
    name='amlcluster1',
    type='amlcompute',
    size='STANDARD_D2_V2',
    min_instances=0,
    max_instances=1,
    tier='LowPriority'
)

compute_cluster=ml_client.compute.begin_create_or_update(cluster)

In [38]:
src_folder='scripts'
import os 
os.makedirs(src_folder,exist_ok=True)

In [65]:
%%writefile $src_folder/conda.yaml

name: environment
channels:
     - conda-forge
dependencies:
     - python=3.8
     - pandas
     - scikit-learn
     - numpy
     - matplotlib

     

Overwriting scripts/conda.yaml


In [66]:
from azure.ai.ml.entities import Environment
env=Environment(
    name='env',
    image='mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04',
    conda_file='scripts/conda.yaml',
    description='custom_environment'
)


In [67]:
ml_client.environments.create_or_update(env)

Environment({'intellectual_property': None, 'is_anonymous': False, 'auto_increment_version': False, 'auto_delete_setting': None, 'name': 'env', 'description': 'custom_environment', 'tags': {}, 'properties': {'azureml.labels': 'latest'}, 'print_as_yaml': True, 'id': '/subscriptions/7b1b43ca-4b64-43cf-9446-edb35a04d7d1/resourceGroups/rg01databricks/providers/Microsoft.MachineLearningServices/workspaces/ws-azureml-01/environments/env/versions/2', 'Resource__source_path': None, 'base_path': '/mnt/batch/tasks/shared/LS_root/mounts/clusters/satyakebakshi951/code/src', 'creation_context': <azure.ai.ml.entities._system_data.SystemData object at 0x7f665018c580>, 'serialize': <msrest.serialization.Serializer object at 0x7f665018c250>, 'version': '2', 'latest_version': None, 'conda_file': {'channels': ['conda-forge'], 'dependencies': ['python=3.8', 'pandas', 'scikit-learn', 'numpy', 'matplotlib'], 'name': 'environment'}, 'image': 'mcr.microsoft.com/azureml/openmpi4.1.0-ubuntu20.04', 'build': None

In [74]:
from azure.ai.ml import command
job_command=command(
    display_name='diabetes-on-computecluster',
    code='./scripts',
    command='python training_script.py --input_directory diabetes.csv --max_depth=3 --min_samples=3 --test_size=0.1 --random_state=41',
    compute='amlcluster1',
    environment='env')
submitted_job=ml_client.create_or_update(job_command)
submitted_job.studio_url

Class AutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class AutoDeleteConditionSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseAutoDeleteSettingSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class IntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class ProtectionLevelSchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Class BaseIntellectualPropertySchema: This is an experimental class, and may change at any time. Please see https://aka.ms/azuremlexperimental for more information.
Uploading scripts (0.0 MBs): 100%|████

Exception: 
[37m
[30m
1) At least one required parameter is missing[39m[39m

Details: 

[31m(x) Failed to extract version when parsing asset env of type environments as arm id. Version must be provided.[39m

Resolutions: 
1) Ensure all parameters required by the Asset schema are specified.
If using the CLI, you can also check the full log in debug mode for more details by adding --debug to the end of your command

Additional Resources: The easiest way to author a yaml specification file is using IntelliSense and auto-completion Azure ML VS code extension provides: [36mhttps://code.visualstudio.com/docs/datascience/azure-machine-learning.[39m To set up VS Code, visit [36mhttps://docs.microsoft.com/azure/machine-learning/how-to-setup-vs-code[39m
